# Accessing Line Task 2014 data

### 20151124

This notebook shows how to access the data in Alldata/fnames.mat.

## Data import

Import latency and error data which is computed from the raw data by
the Octave script lt_analyse_all.m. This script saves the resulting
data into a Matlab v7 file.

(20151120 update) The data saved into the file is as follows:

Note that the analysis of the raw data has the idea of an "event". An
event is the change in position of a distractor line or the target
line. For the no-distractor condition, only target events need be
processed. In the synchronous distractor condition, the distractor
events are not handled separately; a distractor error is the same
thing in this case as a target direction error. In the asynchronous
case, there *are* these two types of error - target direction and
"following the distractor by mistake".

* **r_omitted** A list of the events which were omitted from analysis. These will mostly be events before which the stylus did not settle to a stable position. Also, target events where the target moves a negligible distance are also generally omitted. I've decided not to omit any outliers in the octave code, but events that seem toofast (<100 ms latency) ARE omitted.
* **r_latency** The latency of all non-omitted events.
* **r_latency_error** The latency of all events which have an error
* **r_latency_error_target** The latency of all target events which have an error
* **r_latency_error_distractor** The latency of all distractor events which have an error (async condition only)
* **r_latency_noerror** The latency of all events which have no error
* **r_latency_noerror_target** The latency of all target events which have no error
* **r_latency_noerror_distractor** The latency of all distractor events which have no error (async condition only)
* **r_firstmove_endpoint** The distance from the target to the stylus after the first smooth movement - i.e. how accurate is the first movement? Targets only.



In [1]:
# Import the data, which should be available in Matlab v7 format:
import scipy.io as sio
mat_workspace = sio.loadmat('AllData/fnames.mat')
# fnames is used throughout the rest of this notebook, so this section needs to be evaluated!
fnames = mat_workspace['fnames']

## About the data
### Indexing into the latency data
We have the data now, so lets review how to index into it:

In [3]:
index = 31 # Each index is an individual trial; there are three indices per subject
print 'Data index', index
print 'Data identifier', fnames[0,index][0]

print '_All_ events latencies:'
print fnames[2,index][0]

print 'No error latencies:'
print fnames[3,index]

print 'No error latencies (target):'
print fnames[4,index]

print 'No error latencies (distractor):'
print fnames[5,index]

print 'Error latencies:'
print fnames[6,index]

print 'Error latencies (target):'
print fnames[7,index]

print 'Error latencies (distractor):'
print fnames[8,index]

print 'First motion distance:'
print fnames[9,index]

print 'Parameters for the run', fnames[1,index]

Data index 31
Data identifier Katie/JS/line/20141117163702.txt
_All_ events latencies:
[  1.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e+00   1.60000000e+01   0.00000000e+00
   7.18000000e+02   1.72800000e+03]
No error latencies:
[[  6.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
    2.30000000e+02   8.10000000e+01   3.16000000e+02   1.05700000e+03]
 [  8.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
    1.45000000e+02   5.96000000e+02   1.60000000e+01   1.04100000e+03]
 [  9.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
    4.11000000e+02   1.01900000e+03  -3.62000000e+02   6.79000000e+02]
 [  1.30000000e+01   1.00000000e+00   0.00000000e+00   1.00000000e+00
    2.75000000e+02   3.20000000e+02  -3.85000000e+02   1.10600000e+03]
 [  1.60000000e+01   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.80000000e+02   7.00000000e+01  -2.21000000e+02   8.85000000e+02]
 [  2.30000000

### Latency elements are arrays

Note that the latency elements are themselves arrays (Numpy ndarrays),
containing event index and latency in milliseconds

In [3]:
index = 3
print 'Error events latencies:'
print fnames[6,index]
print 'All latency tables except for fnames[2,index] have following columns:'
print '---------------------------------------------------------'
print 'Event number | type | move error | correct_move | latency'
print '---------------------------------------------------------'
print 'where type is 0 for distractors, 1 for targets'

print 'Event', fnames[6,index][0,0], 'has latency', fnames[6,index][0,4]
print 'Event', fnames[6,index][1,0], 'has latency', fnames[6,index][1,4]

Error events latencies:
[[   6.    1.    1.    0.  125.]
 [  27.    1.    1.    0.  164.]]
All latency tables have following columns:
---------------------------------------------------------
Event number | type | move error | correct_move | latency
---------------------------------------------------------
where type is 0 for distractors, 1 for targets
Event 6.0 has latency 125.0
Event 27.0 has latency 164.0


### Parameters object

The parameters object for each trial has a number of pertinant
entries, packaged up within an array. Unfortunately, the way these are
packaged in octave, where you could refer to "params.tAtteso" doesn't
carry over into this environment and as a result we have some slightly
odd indexing to do:

In [4]:
index = 100
print 'Selected parameters from trial index:', index
print 'params element 0: Duration:',fnames[1,index][0,0][0][0,0]
print 'params element 10: tAtteso (jump time)',fnames[1,index][0,0][10][0,0]
print 'params element 36: Condition description:',fnames[1,index][0,0][36][0]

Selected parameters from trial index: 100
params element 0: Duration: 120.0
params element 10: tAtteso (jump time) 0.8
params element 36: Condition description: No Distractor


### Looping through the data

Here's how to loop through the data. Note that fnames is transposed
here with the zip call:

In [5]:
counter = 0
for fname in zip(*fnames):
    counter = counter + 1
    print fname[0][0], 'is a', fname[1][0,0][36][0], 'trial'
    if counter > 10:
        print '...etc.'
        break

Jon/EF/line/20141118134208.txt is a Synchronous Distractor trial
Jon/EF/line/20141118133619.txt is a No Distractor trial
Jon/EF/line/20141118133914.txt is a Asynchronous Distractor trial
Jon/SF/line/20141125145758.txt is a No Distractor trial
Jon/SF/line/20141125145535.txt is a Synchronous Distractor trial
Jon/SF/line/20141125145313.txt is a Asynchronous Distractor trial
Jon/EJ/line/20141201164116.txt is a Asynchronous Distractor trial
Jon/EJ/line/20141201164343.txt is a No Distractor trial
Jon/EJ/line/20141201164607.txt is a Synchronous Distractor trial
Jon/EC2/line/20141118132305.txt is a Asynchronous Distractor trial
Jon/EC2/line/20141118132030.txt is a Synchronous Distractor trial
...etc.


## Check parameters

The following makes a check of the data to ensure that the parameters
which the experimenter set on the tablet are all correct, in
particular the duration of the task and the mean time between target
jumps.

In [0]:
index = 0
for fname in zip(*fnames):
    problem = 0
    if fname[1][0,0][0][0,0] != 120:
        print fname[0], ' does not have duration parameter = 120s! PROBLEM'
        problem = 1
    if fname[1][0,0][10][0,0] != 0.8:
        print fname[0], ' does not have tAtteso (jump time) parameter = 0.8s! PROBLEM'
        problem = 1
    if problem == 0:
        print 'Trial',index,fname[0][0], 'is a', fname[1][0,0][36][0], 'trial'
    index = index + 1
    if index > 10:
        print '...etc.'
        break

Trial 0 Jon/EF/line/20141118134208.txt is a Synchronous Distractor trial
Trial 1 Jon/EF/line/20141118133619.txt is a No Distractor trial
Trial 2 Jon/EF/line/20141118133914.txt is a Asynchronous Distractor trial
Trial 3 Jon/SF/line/20141125145758.txt is a No Distractor trial
Trial 4 Jon/SF/line/20141125145535.txt is a Synchronous Distractor trial
Trial 5 Jon/SF/line/20141125145313.txt is a Asynchronous Distractor trial
Trial 6 Jon/EJ/line/20141201164116.txt is a Asynchronous Distractor trial
Trial 7 Jon/EJ/line/20141201164343.txt is a No Distractor trial
Trial 8 Jon/EJ/line/20141201164607.txt is a Synchronous Distractor trial
Trial 9 Jon/EC2/line/20141118132305.txt is a Asynchronous Distractor trial
Trial 10 Jon/EC2/line/20141118132030.txt is a Synchronous Distractor trial
...etc.


### Examine data for particular subject/condition

Here's a utility to look at the numbers we have for a specific subject
and a particular condition.

In [9]:
# From the condition string, return an index for the condition. 0 is 
# "No Distractor trial", 1 is "Synchronous Distractor trail", 2 is
# "Asynchronous Distractor trial".
def getcondition (condition_string):
    condition_index = -1
    if 'No Dist' in condition_string:
        condition_index = 0
    elif 'Synchro' in condition_string:
        condition_index = 1
    elif 'Asynchr' in condition_string:
        condition_index = 2
    return condition_index

# Modify these before running:
subject='SF'
condition=1 # 0 ND, 1 S, 2 AS

# Find the index for subj/condition
index = 0
for fname in zip(*fnames):
    if subject in fname[0][0]:
        print 'got subject condition',getcondition(fname[1][0,0][36][0])
        if condition == getcondition (fname[1][0,0][36][0]):
            break;
    index = index + 1

print 'Data index', index
print 'Data identifier', fnames[0,index][0]
#print 'correct-dirn events mean and SD', fnames[1,index][0,0], fnames[1,index][0,1]
#print 'incorrect-dirn events mean and SD', fnames[2,index]
#print '_All_ events latencies:'
#print fnames[2,index]
print 'No-error direction latencies:'
print fnames[3,index]
print 'Error direction events latencies:'
print fnames[6,index]
print 'Parameters for the run', fnames[1,index]

got subject condition 0
got subject condition 1
Data index 4
Data identifier Jon/SF/line/20141125145535.txt
No-error direction latencies:
[[  1.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.75000000e+02]
 [  2.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.01000000e+02]
 [  3.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.35000000e+02]
 [  4.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.80000000e+02]
 [  6.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.90000000e+02]
 [  1.30000000e+01   1.00000000e+00   0.00000000e+00   1.00000000e+00
    2.90000000e+02]
 [  1.60000000e+01   1.00000000e+00   0.00000000e+00   1.00000000e+00
    2.70000000e+02]
 [  1.70000000e+01   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.44000000e+02]
 [  2.70000000e+01   1.00000000e+00   0.00000000e+00   1.00000000e+00
    3.95000000e+02]
 [  2.80000000e+01   1.00000000e+00   0.00000000e+00